In [1]:
import pandas as pd
import numpy as np

In [2]:
pivot_data4 = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')
smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_97789/1207623996.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')


In [3]:
pivot_data4 = pivot_data4.drop(columns=['motion_type', 'motion_encoded'])

In [4]:
import pandas as pd
import numpy as np

def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']

    # Calculate magnitudes along x, y, and z axes
    accel_mag_x_y = np.sqrt(accel_x**2 + accel_y**2)
    accel_mag_z = np.abs(accel_z)
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check if there is significant upward or downward motion along the z-axis
            if accel_z > 0 and accel_mag_x_y > accel_mag_z:
                motion_type = "Upward Horizontal Motion"
            elif accel_z < 0 and accel_mag_x_y > accel_mag_z:
                motion_type = "Downward Horizontal Motion"
            else:
                motion_type = "Other Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
pivot_data4['motion_type'] = pivot_data4.apply(classify_motion_row, axis=1)


In [5]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,linear_accel_z,orientation_x,orientation_y,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type
0,2017-06-29 07:45:56.262,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
1,2017-06-29 07:46:01.263,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Other Moving
2,2017-06-29 07:46:01.324,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
3,2017-06-29 07:46:01.329,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,-0.033066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
4,2017-06-29 07:46:01.402,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Turning/Tilting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,2017-07-13 20:21:35.009,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
173658,2017-07-13 20:21:35.020,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
173659,2017-07-13 20:21:35.060,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
173660,2017-07-13 20:21:35.124,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary


In [6]:
motion_distribution = pivot_data4['motion_type'].value_counts()
print(motion_distribution)


motion_type
Stationary                    156088
Turning/Tilting                 8986
Downward Horizontal Motion      3376
Upward Horizontal Motion        3213
Other Moving                    1999
Name: count, dtype: int64


In [40]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Create and fit the LabelEncoder
label_encoder_motion = LabelEncoder()
pivot_data4['motion_encoded'] = label_encoder_motion.fit_transform(pivot_data4['motion_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_motion, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_motion.pkl')

# Get the encodings and decodings
encodings = label_encoder_motion.transform(label_encoder_motion.classes_)
decodings = label_encoder_motion.classes_

# Display the encodings and decodings
for encoding, decoding in zip(encodings, decodings):
    print(f"Encoding: {encoding}, Decoding: {decoding}")

Encoding: 0, Decoding: Downward Horizontal Motion
Encoding: 1, Decoding: Other Moving
Encoding: 2, Decoding: Stationary
Encoding: 3, Decoding: Turning/Tilting
Encoding: 4, Decoding: Upward Horizontal Motion


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import regularizers
import numpy as np

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = pivot_data4['motion_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model with L1 regularization
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4['motion_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=20, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

2024-03-23 13:41:37.405432: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
3474/3474 [==============================] - 16s 4ms/step - loss: 1.4191 - accuracy: 0.7923 - val_loss: 0.5964 - val_accuracy: 0.9661
Epoch 2/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.7892 - accuracy: 0.9365 - val_loss: 0.4904 - val_accuracy: 0.9710
Epoch 3/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.6528 - accuracy: 0.9512 - val_loss: 0.4325 - val_accuracy: 0.9698
Epoch 4/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.5966 - accuracy: 0.9567 - val_loss: 0.4054 - val_accuracy: 0.9719
Epoch 5/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.5745 - accuracy: 0.9585 - val_loss: 0.3799 - val_accuracy: 0.9711
Epoch 6/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.5504 - accuracy: 0.9614 - val_loss: 0.3801 - val_accuracy: 0.9722
Epoch 7/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.5267 - accuracy: 0.9620 - val_loss: 0.3610 - val_ac

In [8]:
import numpy as np

# Get the total number of classes
num_classes = len(label_encoder_motion.classes_)

# Iterate over the rows of the DataFrame and assign random encoded labels
for index, row in pivot_data4.iterrows():
    random_encoded_label = np.random.randint(0, num_classes)
    pivot_data4.at[index, 'true_motion'] = random_encoded_label

Get true activity from user - activity label with additional descriptors for motion type?

In [126]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded,predicted_activity,true_motion
0,465,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[0, 8, 12, 21, 2, 14, 20, 10, 17]",1.0
1,466,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,Other Moving,1,"[1, 6, 7, 13, 5, 16, 4, 11, 18, 19, 3]",2.0
2,466,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[0, 8, 12, 21, 2, 14, 20, 10, 17]",4.0
3,466,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[0, 8, 12, 21, 2, 14, 20, 10, 17]",0.0
4,466,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,Turning/Tilting,3,"[9, 15]",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[0, 8, 12, 21, 2, 14, 20, 10, 17]",3.0
173658,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[0, 8, 12, 21, 2, 14, 20, 10, 17]",4.0
173659,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[0, 8, 12, 21, 2, 14, 20, 10, 17]",4.0
173660,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[0, 8, 12, 21, 2, 14, 20, 10, 17]",2.0


In [9]:
smartphone = smartphone.drop(columns = ['activity_type'])
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])

# Group activity_type_full by timestamp and transform it to align with smartphone
activity_type_grouped = activity_type_full.groupby(smartphone['timestamp'])['activity_type'].apply(lambda x: ','.join(x)).reset_index()

# Merge activity_type_grouped with smartphone based on timestamp
smartphone = pd.merge(smartphone, activity_type_grouped, on='timestamp', how='left')

# Print or further process the updated DataFrame
print(smartphone)

smartphone.fillna(0, inplace=True)

# Filter the DataFrame to keep rows where "activity_type" is not equal to 0
merged_data_s_f = smartphone[smartphone['activity_type'] != 0]
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])

# Extract hour and minute from the timestamp and convert to minutes
merged_data_s_f['timestamp'] = merged_data_s_f['timestamp'].dt.hour * 60 + merged_data_s_f['timestamp'].dt.minute

merged_data_s_f_filtered = merged_data_s_f[['timestamp', 'activity_type']]

                     timestamp  step_detector_values  audio_value_1  \
0      2017-06-29 07:45:16.506                   1.0            0.0   
1      2017-06-29 07:45:17.407                   0.0            0.0   
2      2017-06-29 07:45:18.407                   0.0          227.0   
3      2017-06-29 07:45:19.407                   0.0          590.0   
4      2017-06-29 07:45:20.446                   0.0         1724.0   
...                        ...                   ...            ...   
617948 2017-07-13 20:13:33.671                   0.0         1956.0   
617949 2017-07-13 20:13:34.663                   0.0         2673.0   
617950 2017-07-13 20:13:35.663                   0.0         2340.0   
617951 2017-07-13 20:13:36.668                   0.0         2046.0   
617952 2017-07-13 20:13:37.668                   0.0         2441.0   

        audio_value_2  audio_value_3  audio_value_4 activity_type  
0             0.00000            0.0            0.0   Video games  
1          

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_97789/4048118510.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_97789/4048118510.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s_f['timestamp'] = merged_data_s_f['timestamp'].dt.hour * 60 + merged_data_s_f['timestamp'].dt.minute


In [10]:
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
pivot_data4['timestamp'] = pd.to_datetime(pivot_data4['timestamp'])
# Extract hour and minute from the timestamp and convert to minutes
pivot_data4['timestamp'] = pivot_data4['timestamp'].dt.hour * 60 + pivot_data4['timestamp'].dt.minute
# Merge the dataframes on the 'timestamp' column
pivot_data4_merged = pivot_data4.merge(merged_data_s_f_filtered, on='timestamp', how='left')
pivot_data4_merged.fillna(0, inplace=True)
pivot_data4_merged_filtered = pivot_data4_merged[pivot_data4_merged['activity_type'] != 0]

In [25]:
valid_activities = [
        # Downward Horizontal Motion
        'Running', 'Walk', 'Walking & party', 'Football', 'In vehicle', 'Train', 'In bus',
        # Upward Horizontal Motion
        'In vehicle', 'Train', 'In bus', 'Running', 'Walk', 'Walking & party', 'Football',
        # Other Moving
        'Cooking', 'Meeting', 'Movie', 'Shopping& wearing', 'In vehicle', 'Train', 'In bus', 'Running', 'Walk', 'Walking & party', 'Football',
        # Turning/Tilting
        'Phone was out of the pocket (forgot)', 'Took off glasses',
        # Stationary
        'At home', 'Pause', 'Shop', 'Work', 'Eat', 'Sleep', 'Watching TV', 'Picnic', 'Video games', 'In Computer', 'On bus stop'
    ]

In [49]:
from sklearn.preprocessing import LabelEncoder

# Create and fit the LabelEncoder
label_encoder_activities = LabelEncoder()
pivot_data4_merged_filtered['activity_encoded'] = label_encoder_activities.fit_transform(pivot_data4_merged_filtered['activity_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_activities, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_activities.pkl')

# Get unique classes from the LabelEncoder
unique_classes = label_encoder_activities.classes_

# Print unique classes
print("Unique Classes:")
for class_ in unique_classes:
    print(class_)

# Get the encodings and decodings
encodings = label_encoder_activities.transform(label_encoder_activities.classes_)
decodings = label_encoder_activities.classes_

# Display the encodings and decodings
for encoding, decoding in zip(encodings, decodings):
    print(f"Encoding: {encoding}, Decoding: {decoding}")


Unique Classes:
At home
Cooking
Eat
Football
In bus
In computer
In vehicle
Meeting
Movie
On bus stop
Pause
Phone was out of the pocket (forgot)
Picnic 
Running
Shop
Shopping& wearing
Sleep
Took off glasses
Train
Video games
Walk
Walking&party
Watching TV
Work
Encoding: 0, Decoding: At home
Encoding: 1, Decoding: Cooking
Encoding: 2, Decoding: Eat
Encoding: 3, Decoding: Football
Encoding: 4, Decoding: In bus
Encoding: 5, Decoding: In computer
Encoding: 6, Decoding: In vehicle
Encoding: 7, Decoding: Meeting
Encoding: 8, Decoding: Movie
Encoding: 9, Decoding: On bus stop
Encoding: 10, Decoding: Pause
Encoding: 11, Decoding: Phone was out of the pocket (forgot)
Encoding: 12, Decoding: Picnic 
Encoding: 13, Decoding: Running
Encoding: 14, Decoding: Shop
Encoding: 15, Decoding: Shopping& wearing
Encoding: 16, Decoding: Sleep
Encoding: 17, Decoding: Took off glasses
Encoding: 18, Decoding: Train
Encoding: 19, Decoding: Video games
Encoding: 20, Decoding: Walk
Encoding: 21, Decoding: Walking&p

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_97789/3896208718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['activity_encoded'] = label_encoder_activities.fit_transform(pivot_data4_merged_filtered['activity_type'])


In [27]:
valid_activities_encoded = label_encoder_activities.fit_transform(valid_activities)

In [50]:
def classify_activity(row):
    motion_type = row['motion_type']
    if motion_type == "Downward Horizontal Motion":
        return [13, 20, 21, 3, 6, 18, 4]
    elif motion_type == "Upward Horizontal Motion":
        return [6, 18, 4, 13, 20, 21, 3]
    elif motion_type == "Other Moving":
        return [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]
    elif motion_type == "Turning/Tilting":
        return [11, 17]
    elif motion_type == "Stationary":
        return [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]

# Apply the function to create the predicted_activity column
pivot_data4['predicted_activity'] = pivot_data4.apply(classify_activity, axis=1)


In [38]:
valid_activities_encoded[:7]

array([13, 20, 21,  3,  6, 18,  5])

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'motion_encoded', 'timestamp']]
y = pivot_data4['predicted_activity']

# Convert labels into binary format for multi-label classification
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(mlb.classes_), activation='sigmoid')  # Sigmoid activation for multi-label classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=1, callbacks=[EarlyStopping(patience=5)])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Convert predictions back to original labels
y_pred_labels = mlb.inverse_transform(y_pred)

# Convert ground truth labels back to original format
y_true_labels = mlb.inverse_transform(y_test)

# Convert ground truth labels to binary array format
y_true_binary = mlb.transform(y_true_labels)

# Print classification report
print(classification_report(y_true_binary, y_pred, target_names=mlb.classes_.astype(str)))

2024-03-23 18:56:42.643833: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
3474/3474 [==============================] - 16s 4ms/step - loss: 0.0610 - accuracy: 0.1483 - val_loss: 0.0104 - val_accuracy: 0.6865
Epoch 2/10
3474/3474 [==============================] - 13s 4ms/step - loss: 0.0082 - accuracy: 0.1395 - val_loss: 0.0047 - val_accuracy: 0.0557
Epoch 3/10
3474/3474 [==============================] - 12s 4ms/step - loss: 0.0048 - accuracy: 0.1155 - val_loss: 0.0024 - val_accuracy: 0.0223
Epoch 4/10
3474/3474 [==============================] - 12s 3ms/step - loss: 0.0038 - accuracy: 0.0742 - val_loss: 0.0021 - val_accuracy: 0.0542
Epoch 5/10
3474/3474 [==============================] - 13s 4ms/step - loss: 0.0032 - accuracy: 0.0688 - val_loss: 0.0025 - val_accuracy: 0.0519
Epoch 6/10
3474/3474 [==============================] - 12s 3ms/step - loss: 0.0030 - accuracy: 0.0636 - val_loss: 0.0020 - val_accuracy: 0.0528
Epoch 7/10
3474/3474 [==============================] - 12s 3ms/step - loss: 0.0028 - accuracy: 0.0457 - val_loss: 0.0024 - val_ac

In [32]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.metrics import classification_report

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'motion_encoded', 'timestamp']]
y = pivot_data4['predicted_activity']

# Convert labels into binary format for multi-label classification
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

from sklearn.multioutput import MultiOutputClassifier

# Initialize a multi-output classifier with GradientBoostingClassifier as the base estimator
multi_gb_model = MultiOutputClassifier(GradientBoostingClassifier())

# Train the model
multi_gb_model.fit(X_train_scaled, y_train)

# Evaluate the model
test_accuracy_gb = multi_gb_model.score(X_test_scaled, y_test)
print("Test Accuracy (Gradient Boosting):", test_accuracy_gb)

# Make predictions
y_pred_gb = multi_gb_model.predict(X_test_scaled)

# Print classification report
print(classification_report(y_test, y_pred_gb, target_names=mlb.classes_.astype(str)))

# Define the file path to save the model
model_file_path = "/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/multi_gb_model.pkl"

# Save the model
joblib.dump(multi_gb_model, model_file_path)

print("Model saved successfully at:", model_file_path)



Test Accuracy (Gradient Boosting): 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31243
           1       1.00      1.00      1.00       398
           2       1.00      1.00      1.00     31243
           3       1.00      1.00      1.00      1715
           4       1.00      1.00      1.00     31243
           5       1.00      1.00      1.00      1715
           6       1.00      1.00      1.00      1715
           7       1.00      1.00      1.00       398
           8       1.00      1.00      1.00       398
           9       1.00      1.00      1.00     31243
          10       1.00      1.00      1.00     31243
          11       1.00      1.00      1.00      1775
          12       1.00      1.00      1.00     31243
          13       1.00      1.00      1.00      1715
          14       1.00      1.00      1.00     31243
          15       1.00      1.00      1.00       398
          16       1.00      1.00      1.0

Interestingly, gradient boost performs better than the neural network here. Can the approaches potentially be combined depending on where either model architecture performs best?

This diversity of motion_types shows that the function is working. 

The logic I have now is that smartphone and smartwatch etc. indeed do not have been used simultaneously. Regardless, we may get some insights regarding the timing of activities and filter accordingly when excluding day and year. 

We can now predict activities with this time context. With true labels given by users the model can learn iteratively. 

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# Load the previously trained model
model = load_model("/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras")

# Define the new target variable (y) using single true labels
y_single_true_labels = pivot_data4_merged_filtered['activity_encoded']

# Convert labels into binary format for single-label classification
label_binarizer = LabelBinarizer()
y_single_true_binary = label_binarizer.fit_transform(y_single_true_labels)

# Filter X to include only rows with true labels
X_filtered = X.loc[y_single_true_labels.index]

# Split the filtered data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_single_true_binary, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Retrain the model with class weights
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, verbose=1, callbacks=[EarlyStopping(patience=5)])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

# Print classification report
print(classification_report(y_test, y_pred, target_names=label_binarizer.classes_.astype(str)))

Epoch 1/20
246/246 [==============================] - 4s 5ms/step - loss: 4.4315 - accuracy: 0.1419 - val_loss: 0.3705 - val_accuracy: 0.1770
Epoch 2/20
246/246 [==============================] - 1s 4ms/step - loss: 0.4111 - accuracy: 0.1855 - val_loss: 0.2403 - val_accuracy: 0.1882
Epoch 3/20
246/246 [==============================] - 1s 4ms/step - loss: 0.1966 - accuracy: 0.1911 - val_loss: 0.2012 - val_accuracy: 0.1872
Epoch 4/20
246/246 [==============================] - 2s 6ms/step - loss: 0.1743 - accuracy: 0.1858 - val_loss: 0.1833 - val_accuracy: 0.1882
Epoch 5/20
246/246 [==============================] - 1s 3ms/step - loss: 0.1649 - accuracy: 0.1943 - val_loss: 0.1704 - val_accuracy: 0.1877
Epoch 6/20
246/246 [==============================] - 1s 4ms/step - loss: 0.1595 - accuracy: 0.1964 - val_loss: 0.1647 - val_accuracy: 0.1887
Epoch 7/20
246/246 [==============================] - 1s 4ms/step - loss: 0.1563 - accuracy: 0.1962 - val_loss: 0.1579 - val_accuracy: 0.1902
Epoch 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn

In [33]:
model = joblib.load("/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/multi_gb_model.pkl")

# Define the new target variable (y) using single true labels
y_single_true_labels = pivot_data4_merged_filtered['activity_encoded']

# Convert labels into binary format for single-label classification
label_binarizer = LabelBinarizer()
y_single_true_binary = label_binarizer.fit_transform(y_single_true_labels)

# Filter X to include only rows with true labels
X_filtered = X.loc[y_single_true_labels.index]

# Split the filtered data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_single_true_binary, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
multi_gb_model.fit(X_train_scaled, y_train)

# Evaluate the model
test_accuracy_gb = multi_gb_model.score(X_test_scaled, y_test)
print("Test Accuracy (Gradient Boosting):", test_accuracy_gb)

# Make predictions
y_pred_gb = multi_gb_model.predict(X_test_scaled)

# Print classification report
print(classification_report(y_test, y_pred_gb, target_names=mlb.classes_.astype(str)))

# Define the file path to save the model
model_file_path = "/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/multi_gb_model.pkl"

# Save the model
joblib.dump(multi_gb_model, model_file_path)

print("Model saved successfully at:", model_file_path)

Test Accuracy (Gradient Boosting): 0.001628001628001628
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       141
           1       0.00      0.00      0.00        73
           2       1.00      0.00      0.01       243
           3       0.00      0.00      0.00        37
           4       0.43      0.01      0.01       458
           5       0.00      0.00      0.00       173
           6       0.00      0.00      0.00       135
           7       0.00      0.00      0.00       116
           8       0.00      0.00      0.00       119
           9       0.00      0.00      0.00        14
          10       0.00      0.00      0.00       112
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00        16
          14       0.00      0.00      0.00        71
          15       0.00      0.00      0.00        32
          16       0.00  

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn

In [37]:
pivot_data4_merged_filtered

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded,true_motion,activity_type,activity_encoded
0,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0.0,Video games,19
1,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0.0,In computer,5
2,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0.0,At home,0
3,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0.0,In computer,5
4,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0.0,Eat,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129174,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0.0,Cooking,1
129175,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0.0,Video games,19
129176,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0.0,In bus,4
129177,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0.0,In vehicle,6


In [69]:
def classify_motion_activity(row):
    activity_encoded = row['activity_encoded']
    if activity_encoded in [13, 20, 21, 3, 6, 18, 4]:
        true_motion = 0
    elif activity_encoded in [6, 18, 4, 13, 20, 21, 3]:
        true_motion = 4
    elif activity_encoded in [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]:
        true_motion = 1
    elif activity_encoded in [11, 17]:
        true_motion = 3
    elif activity_encoded in [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]:
        true_motion = 2
    return true_motion

# Assuming 'pivot_data4' is your DataFrame
pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_97789/179481233.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)


In [53]:
pivot_data4_merged_filtered

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded,true_motion,activity_type,activity_encoded
0,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,2,Video games,19
1,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,2,In computer,5
2,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,2,At home,0
3,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,2,In computer,5
4,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,2,Eat,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129174,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,1,Cooking,1
129175,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,2,Video games,19
129176,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,In bus,4
129177,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,In vehicle,6


In [72]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model("/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras")

X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]

# Define the new target variable (y) using single true labels
y = pivot_data4_merged_filtered['true_motion']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Encode class labels for the new target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with class weights and early stopping
history = loaded_model.fit(X_train_scaled, y_train_encoded, epochs=100, batch_size= 45, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict, callbacks=[early_stopping])

# Evaluate the retrained model on the test set
test_loss, test_accuracy = loaded_model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = loaded_model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)



Epoch 1/100
175/175 [==============================] - 3s 6ms/step - loss: 2.9984 - accuracy: 0.2154 - val_loss: 1.9227 - val_accuracy: 0.3286
Epoch 2/100
175/175 [==============================] - 1s 4ms/step - loss: 1.9402 - accuracy: 0.2314 - val_loss: 1.6721 - val_accuracy: 0.3413
Epoch 3/100
175/175 [==============================] - 1s 5ms/step - loss: 1.7909 - accuracy: 0.2580 - val_loss: 1.6162 - val_accuracy: 0.3505
Epoch 4/100
175/175 [==============================] - 1s 7ms/step - loss: 1.7063 - accuracy: 0.2559 - val_loss: 1.6593 - val_accuracy: 0.3474
Epoch 5/100
175/175 [==============================] - 1s 3ms/step - loss: 1.7183 - accuracy: 0.2349 - val_loss: 1.5906 - val_accuracy: 0.3474
Epoch 6/100
175/175 [==============================] - 1s 4ms/step - loss: 1.6904 - accuracy: 0.2337 - val_loss: 1.6070 - val_accuracy: 0.3266
Epoch 7/100
175/175 [==============================] - 1s 4ms/step - loss: 1.6219 - accuracy: 0.2249 - val_loss: 1.5873 - val_accuracy: 0.3489